In [1]:
import exmel
import pickle

In [5]:
dataset = exmel.Dataset("dataset_v1")

In [2]:
with open("song_name_to_candidates.pkl", "rb") as f:
    song_name_to_candidates = pickle.load(f)

In [6]:
from tqdm.auto import tqdm
from exmel.sequence import song_stats

score_model = exmel.XGBoostModel('xgb_hop1_miss2_len8_large.json')
name_to_scores: dict[str, list[float]] = {}
for song in tqdm(dataset):
    score_model.load_song_stats(song_stats(song.melody, song.performance))
    candidates = song_name_to_candidates[song.name]
    name_to_scores[song.name] = score_model(candidates)

  0%|          | 0/34 [00:00<?, ?it/s]

In [7]:
from itertools import product
from exmel.wisp import weighted_interval_scheduling
from exmel.alignment import Alignment, concat_matches, FrozenMatch
import numpy as np


pen_min_tuple = list(product([0], range(31)))

f1_list: list[float] = []
precision_list: list[float] = []
recall_list: list[float] = []

for p, m in pen_min_tuple:
    f1_score = 0
    precision = 0
    recall = 0
    for song in tqdm(dataset, desc=f"{p=}, {m=}"):
        scores = np.array(name_to_scores[song.name]) - p
        candidates = song_name_to_candidates[song.name]
        updated_candidates: list[FrozenMatch] = []
        for candidate, score in zip(candidates, scores):
            if score >= m:
                updated_candidates.append(candidate.update_score(score))
        candidates = updated_candidates
        opt_score, opt_subset = weighted_interval_scheduling(
            candidates, return_subset=True, verbose=False)
        discarded_matches = [match for match in candidates if match not in opt_subset]
        concat_events = concat_matches(opt_subset)
        alignment = Alignment(concat_events, opt_subset, discarded_matches, opt_score,
                        sum(match.sum_miss for match in opt_subset),
                        sum(match.sum_error for match in opt_subset))
        assert song.ground_truth is not None
        result = exmel.evaluate_melody(song.ground_truth, alignment.events, plot=False)
        f1_score += result.f1_score
        precision += result.precision
        recall += result.recall
    f1_score /= len(dataset)
    precision /= len(dataset)
    recall /= len(dataset)
    f1_list.append(f1_score)
    precision_list.append(precision)
    recall_list.append(recall)
    print(f"{p=}, {m=}, {f1_score=}, {precision=}, {recall=}")

p=0, m=0:   0%|          | 0/34 [00:00<?, ?it/s]

p=0, m=0, f1_score=0.9366007610850913, precision=0.9330983836544795, recall=0.9409448372095421


p=0, m=1:   0%|          | 0/34 [00:00<?, ?it/s]

p=0, m=1, f1_score=0.939840474391724, precision=0.9394254901163056, recall=0.9409448372095421


p=0, m=2:   0%|          | 0/34 [00:00<?, ?it/s]

...

p=0, m=28, f1_score=0.9199533738852507, precision=0.9807301936627605, recall=0.871633298565883


p=0, m=29:   0%|          | 0/34 [00:00<?, ?it/s]

p=0, m=29, f1_score=0.9189976259189181, precision=0.981431263455327, recall=0.8700583230292888


p=0, m=30:   0%|          | 0/34 [00:00<?, ?it/s]

p=0, m=30, f1_score=0.9170750753513791, precision=0.9821234706890118, recall=0.8672721136462793


# Data

In [12]:
from dataclasses import dataclass

@dataclass(frozen=True, slots=True)
class Detection:
    tp: int
    fp: int
    start: float
    end: float

with open("detections.pkl", "rb") as f:
    detections = pickle.load(f)

In [13]:
from typing import TypedDict
from exmel.alignment import MatchLike
import numpy as np
from collections import Counter

class Entry(TypedDict):
    name: str
    tp: int
    fp: int
    length: int
    misses: int
    error: float
    velocity: float
    duration: float
    note_mean: float
    note_std: float
    note_entropy: float
    note_unique: int
    note_change: int

class ScoreFunctions:

    @staticmethod
    def length(match: MatchLike) -> int:
        return len(match.events)

    @staticmethod
    def misses(match: MatchLike) -> int:
        return match.sum_miss

    @staticmethod
    def error(match: MatchLike) -> float:
        return match.sum_error

    @staticmethod
    def velocity(match: MatchLike) -> float:
        return float(np.mean([event.velocity for event in match.events]))
    
    @staticmethod
    def duration(match: MatchLike) -> float:
        return match.events[-1].time - match.events[0].time

    @staticmethod
    def note_mean(match: MatchLike) -> float:
        return sum(event.note for event in match.events) / len(match.events)

    @staticmethod
    def note_std(match: MatchLike) -> float:
        return float(np.std([event.note for event in match.events]))

    @staticmethod
    def note_entropy(match: MatchLike) -> float:
        counts = np.array(list(Counter(event.note for event in match.events).values()))
        probs = counts / counts.sum()
        entropy = -np.sum(probs * np.log2(probs))
        return float(entropy)

    @staticmethod
    def note_unique(match: MatchLike) -> int:
        return len(set(event.note for event in match.events))

    @staticmethod
    def note_change(match: MatchLike) -> int:
        return sum(match.events[i].note != match.events[i - 1].note
                   for i in range(1, len(match.events)))

sf = ScoreFunctions

In [ ]:
dataset = exmel.Dataset("dataset_v1")


In [19]:
len(detections['All of Me(John Legend)']), len(song_name_to_candidates['All of Me(John Legend)'])

(89455, 89455)

In [21]:
detections['All of Me(John Legend)'][0]

Detection(tp=8, fp=0, start=17.115885416666668, end=19.973958333333332)

In [22]:
from tqdm.auto import tqdm

data: list[Entry] = []

for i, song in enumerate(dataset, 1):
    print(f"{i}/{len(dataset)}")
    for match, dt in tqdm(
        zip(song_name_to_candidates[song.name], detections[song.name]),
        total=len(song_name_to_candidates[song.name])):
        assert song.ground_truth is not None
        tp, fp = dt.tp, dt.fp
        data.append(Entry(
            name=song.name,
            tp=tp,
            fp=fp,
            length=sf.length(match),
            misses=sf.misses(match),
            error=sf.error(match),
            velocity=sf.velocity(match),
            duration=sf.duration(match),
            note_mean=sf.note_mean(match),
            note_std=sf.note_std(match),
            note_entropy=sf.note_entropy(match),
            note_unique=sf.note_unique(match),
            note_change=sf.note_change(match)))

1/34


  0%|          | 0/89455 [00:00<?, ?it/s]

2/34


  0%|          | 0/42679 [00:00<?, ?it/s]

3/34


...

  0%|          | 0/50815 [00:00<?, ?it/s]

33/34


  0%|          | 0/45451 [00:00<?, ?it/s]

34/34


  0%|          | 0/160851 [00:00<?, ?it/s]

In [24]:
import pandas as pd
df = pd.DataFrame(data)
# df.to_csv("matches.csv", index=False, encoding="utf-8")

In [26]:
with open("hop1_miss2_len8_matches.pkl", "wb") as f:
    pickle.dump(data, f)